In [ ]:
import os
import sqlite3
import pandas as pd

# Use the same path as your app / .env
DB_PATH = os.getenv(
    "NBA_EDGE_DB_PATH",
    r"C:\Users\U\OneDrive\NBA_Edge\model_logs.db"  # update YourUser if needed
)

print("DB path:", DB_PATH)
print("Exists:", os.path.exists(DB_PATH))


In [11]:
conn = sqlite3.connect(DB_PATH)

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables:")
print(tables)

df = pd.read_sql("SELECT * FROM logs;", conn)
conn.close()

print("Shape:", df.shape)
df.head()


Tables:
              name
0             logs
1  sqlite_sequence
Shape: (51, 53)


,id,date,favorite,underdog,vegas_line,model_line,edge,pick,confidence,fav_pace,...,dog_player_adj_raw,effective_player_adj,fav_effective_player_adj,dog_effective_player_adj,injury_impact_flag,injury_heavy,fav_injury_impact_flag,dog_injury_impact_flag,fav_injury_heavy,dog_injury_heavy
0,1,2025-11-24,MIA,DAL,-9.0,0.0,18.257327,MIA -9.0,STRONG,NaN,...,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN
1,2,2025-11-25,LAL,LAC,-6.5,0.0,16.166231,LAL -6.5,STRONG,NaN,...,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN
2,6,2025-11-25,ORL,PHI,-4.0,0.0,9.585249,ORL -4.0,STRONG,100.58,...,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN
3,8,2025-11-26,DET,BOS,-2.5,0.0,0.400580,DET -2.5,PASS,101.41,...,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN
4,10,2025-11-26,NYK,CHA,-6.5,0.0,2.821820,CHA +6.5,MEDIUM,100.56,...,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN


In [12]:
def graded_only(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    if "spread_covered" not in out.columns:
        raise ValueError("spread_covered column not found")
    out = out[out["spread_covered"].notna()]
    return out

gdf = graded_only(df)
print("Graded rows:", gdf.shape[0])
gdf.head()


Graded rows: 46


,id,date,favorite,underdog,vegas_line,model_line,edge,pick,confidence,fav_pace,...,dog_player_adj_raw,effective_player_adj,fav_effective_player_adj,dog_effective_player_adj,injury_impact_flag,injury_heavy,fav_injury_impact_flag,dog_injury_impact_flag,fav_injury_heavy,dog_injury_heavy
0,1,2025-11-24,MIA,DAL,-9.0,0.0,18.257327,MIA -9.0,STRONG,NaN,...,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN
1,2,2025-11-25,LAL,LAC,-6.5,0.0,16.166231,LAL -6.5,STRONG,NaN,...,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN
2,6,2025-11-25,ORL,PHI,-4.0,0.0,9.585249,ORL -4.0,STRONG,100.58,...,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN
3,8,2025-11-26,DET,BOS,-2.5,0.0,0.400580,DET -2.5,PASS,101.41,...,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN
4,10,2025-11-26,NYK,CHA,-6.5,0.0,2.821820,CHA +6.5,MEDIUM,100.56,...,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN


In [13]:
def summarize_slice(sub: pd.DataFrame, label: str) -> dict:
    if sub.empty:
        return {
            "Group": label,
            "Picks": 0,
            "Wins": 0,
            "Losses": 0,
            "Pushes": 0,
            "Win%": None,
            "Avg edge (pts)": None,
        }

    N = len(sub)
    W = (sub["spread_covered"] == 1).sum()
    L = (sub["spread_covered"] == 0).sum()
    P = (sub["spread_covered"] == 2).sum()
    effN = N - P

    winp = (W / effN * 100.0) if effN > 0 else None
    avg_e = float(sub["edge"].mean()) if "edge" in sub.columns else None

    return {
        "Group": label,
        "Picks": N,
        "Wins": W,
        "Losses": L,
        "Pushes": P,
        "Win%": round(winp, 1) if winp is not None else None,
        "Avg edge (pts)": round(avg_e, 2) if avg_e is not None else None,
    }


In [14]:
dfe = gdf.copy()
dfe["abs_edge"] = dfe["edge"].abs()

buckets = [
    summarize_slice(dfe[dfe["abs_edge"] < 2], "[0, 2)"),
    summarize_slice(
        dfe[(dfe["abs_edge"] >= 2) & (dfe["abs_edge"] < 4)],
        "[2, 4)",
    ),
    summarize_slice(
        dfe[(dfe["abs_edge"] >= 4) & (dfe["abs_edge"] < 6)],
        "[4, 6)",
    ),
    summarize_slice(dfe[dfe["abs_edge"] >= 6], ">= 6"),
]

edge_bucket_df = pd.DataFrame(buckets)
edge_bucket_df


,Group,Picks,Wins,Losses,Pushes,Win%,Avg edge (pts)
0,"[0, 2)",6,1,5,0,16.7,0.99
1,"[2, 4)",15,9,6,0,60.0,3.09
2,"[4, 6)",8,7,1,0,87.5,4.92
3,>= 6,17,5,12,0,29.4,9.84


In [15]:
if "injury_heavy" in gdf.columns:
    rows = [
        summarize_slice(gdf[gdf["injury_heavy"] == 1], "Injury heavy = 1"),
        summarize_slice(gdf[gdf["injury_heavy"] != 1], "Not heavy (0)"),
    ]
    injury_df = pd.DataFrame(rows)
    injury_df
else:
    print("injury_heavy column not found in logs.")


In [2]:
import sqlite3
import pandas as pd

DB_PATH = r"C:\Users\Arkos\OneDrive\NBA_Edge\model_logs.db"  # adjust if yours is different

conn = sqlite3.connect(DB_PATH)

query = """
SELECT
    date,
    favorite,
    underdog,
    vegas_line,
    pick,
    confidence,
    edge,
    models_aligned,
    spread_covered,
    injury_heavy,
    fav_injury_heavy,
    dog_injury_heavy,
    injury_sum_abs_diff,
    injury_sum_abs_diff_weighted,
    fav_sum_abs_diff,
    dog_sum_abs_diff,
    inj_small_sample,
    fav_small_sample,
    dog_small_sample
FROM logs
"""
df = pd.read_sql_query(query, conn)
conn.close()

# Basic cleaning
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["edge"] = pd.to_numeric(df["edge"], errors="coerce")
df["spread_covered"] = pd.to_numeric(df["spread_covered"], errors="coerce")
for col in [
    "injury_sum_abs_diff",
    "injury_sum_abs_diff_weighted",
    "fav_sum_abs_diff",
    "dog_sum_abs_diff",
]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

for col in ["inj_small_sample", "fav_small_sample", "dog_small_sample"]:
    if col in df.columns:
        df[col] = df[col].fillna(0).astype(int)

df.head()


C:\Users\Arkos\AppData\Local\Temp\ipykernel_18712\646876805.py:48: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(0).astype(int)
C:\Users\Arkos\AppData\Local\Temp\ipykernel_18712\646876805.py:48: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(0).astype(int)
C:\Users\Arkos\AppData\Local\Temp\ipykernel_18712\646876805.py:48: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to th

,date,favorite,underdog,vegas_line,pick,confidence,edge,models_aligned,spread_covered,injury_heavy,fav_injury_heavy,dog_injury_heavy,injury_sum_abs_diff,injury_sum_abs_diff_weighted,fav_sum_abs_diff,dog_sum_abs_diff,inj_small_sample,fav_small_sample,dog_small_sample
0,2025-11-24,MIA,DAL,-9.0,MIA -9.0,STRONG,18.257327,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,2025-11-25,LAL,LAC,-6.5,LAL -6.5,STRONG,16.166231,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,2025-11-25,ORL,PHI,-4.0,ORL -4.0,STRONG,9.585249,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,2025-11-26,DET,BOS,-2.5,DET -2.5,PASS,0.400580,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,2025-11-26,NYK,CHA,-6.5,CHA +6.5,MEDIUM,2.821820,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [3]:
# Only graded picks (W/L, ignore pushes and nulls)
gdf = df[df["spread_covered"].isin([0, 1])].copy()

# Convenience columns
gdf["win"] = (gdf["spread_covered"] == 1).astype(int)
gdf["loss"] = (gdf["spread_covered"] == 0).astype(int)

summary_small_sample = (
    gdf.groupby("inj_small_sample")
    .agg(
        Picks=("spread_covered", "size"),
        Wins=("win", "sum"),
        Losses=("loss", "sum"),
        Avg_edge=("edge", "mean"),
        Avg_injury_sum_abs_diff=("injury_sum_abs_diff", "mean"),
    )
    .reset_index()
)

summary_small_sample["Win%"] = (summary_small_sample["Wins"] /
                                summary_small_sample["Picks"] * 100)

summary_small_sample


,inj_small_sample,Picks,Wins,Losses,Avg_edge,Avg_injury_sum_abs_diff,Win%
0,0,51,24,27,5.773851,NaN,47.058824


In [4]:
def bucket_injury_sum(x: float) -> str:
    if pd.isna(x):
        return "No injuries"
    if x < 5:
        return "[0, 5)"
    if x < 10:
        return "[5, 10)"
    if x < 20:
        return "[10, 20)"
    return ">= 20"

gdf["inj_sum_bucket"] = gdf["injury_sum_abs_diff"].apply(bucket_injury_sum)

summary_inj_bucket = (
    gdf.groupby("inj_sum_bucket")
    .agg(
        Picks=("spread_covered", "size"),
        Wins=("win", "sum"),
        Losses=("loss", "sum"),
        Avg_edge=("edge", "mean"),
        Avg_injury_sum_abs_diff=("injury_sum_abs_diff", "mean"),
    )
    .reset_index()
)

summary_inj_bucket["Win%"] = (summary_inj_bucket["Wins"] /
                              summary_inj_bucket["Picks"] * 100)

summary_inj_bucket.sort_values("Avg_injury_sum_abs_diff")


,inj_sum_bucket,Picks,Wins,Losses,Avg_edge,Avg_injury_sum_abs_diff,Win%
0,No injuries,51,24,27,5.773851,NaN,47.058824
